In [1]:
import numpy as np
import torch as t
from torch.autograd import Variable
import matplotlib.pyplot as plt

#input the saturated pressures
p = np.array([[28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5]])
p = t.tensor(p, requires_grad=False, dtype=t.float32)
p_sat_w = 10.0 ** ( 8.07131 - 1730.63 / (20.0 + 233.426))
p_sat_d = 10.0 ** (7.43155 - 1554.679 / (20.0 + 240.337))

#input the fractions
X1 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] 
X2 = [1.0,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0.0]
x1 = t.tensor(X1, requires_grad=False, dtype=t.float32)
x2 = t.tensor(X2, requires_grad=False, dtype=t.float32)

A = Variable(t.tensor([1.0, 1.0]), requires_grad=True)
a = np.array(([[8.07131, 1730.63, 233.426], [7.43155, 1554.679, 240.337]]))

#ideally we should use armijo line search for alpha
a = 0.0005

for i in range(100):
    #calculate the predicted pressure
    p_pred = x1*t.exp(A[0]*(A[1]*x2/(A[0]*x1+A[1]*x2))**2)*p_sat_w + x2*t.exp(A[1]*(A[0]*x1/(A[0]*x1+A[1]*x2))**2)*p_sat_d

    loss = (p_pred - p)**2 #objective
    loss = loss.sum()
    loss.backward() #calculate loss for every perimeter

    with t.no_grad():
        A -= a * A.grad #substract and assign the value to the variable
        A.grad.zero_() #restart loop

print ('Estimated A12 and A21:',A)
print ('\n')
print ('Objective Value:',loss.data.numpy())
print ('\n')

Estimated A12 and A21: tensor([1.9578, 1.6897], requires_grad=True)


Objective Value: 0.67022634




In [14]:
for i in range (11) :
    x_1 = X1[i]
    x_2= X2[i]
    p_pred = x1*t.exp(A[0]*(A[1]*x2/(A[0]*x1+A[1]*x2))**2)*p_sat_w + x2*t.exp(A[1]*(A[0]*x1/(A[0]*x1+A[1]*x2))**2)*p_sat_d
    
print ('Predicted Pressure', p_pred,1) 
print ('\n')
print('Actual Pressure',p)
print ('\n')
print('Our model fits well with the data')

Predicted Pressure tensor([28.8241, 34.6417, 36.4511, 36.8660, 36.8728, 36.7487, 36.3898, 35.3854,
        32.9501, 27.7333, 17.4733], grad_fn=<AddBackward0>) 1


Actual Pressure tensor([[28.1000, 34.4000, 36.7000, 36.9000, 36.8000, 36.7000, 36.5000, 35.4000,
         32.9000, 27.7000, 17.5000]])


Our model fits well with the data
